In [0]:
# connecting to Azure blob
dbutils.fs.mount(
    source = "wasbs://raw@storageblob0101.blob.core.windows.net",
    mount_point = "/mnt/raw2",
    extra_configs={"fs.azure.account.key.storageblob0101.blob.core.windows.net":"cEjCAg/kXc2SfVO/8LkE77Q4yXYhmmNq1fdFkZaf6YDM77v/YpvTwZVesyr6NXr4ZYSIEpHOtjJH+AStTCApdA=="})

In [0]:
from pyspark.sql.functions import *
import urllib
# reading the sales data from a CSV file
df = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/raw2/sales_data.csv")
df.show()

+-----------+---------------+-----+---------------+----------------+-------+-----+--------+----+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+--------+----------+---------+---------+---------------+----------------+--------+
|ORDERNUMBER|QUANTITYORDERED|PRICE|ORDERLINENUMBER|       ORDERDATE| STATUS|TR_ID|MONTH_ID|YEAR|PRODUCTLINE|MSRP|PRODUCTCODE|        CUSTOMERNAME|           PHONE|        ADDRESSLINE1|ADDRESSLINE2|         CITY|   STATE|POSTALCODE|  COUNTRY|TERRITORY|CONTACTLASTNAME|CONTACTFIRSTNAME|DEALSIZE|
+-----------+---------------+-----+---------------+----------------+-------+-----+--------+----+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+--------+----------+---------+---------+---------------+----------------+--------+
|      10107|             30| 95.7|              2|  2/24/2003 0:00|Shipped|    1|       2|2003|Motorcycles|  95|   S1

In [0]:
# calculating the total revenue
df2 = df.withColumn("Revenue", col("Quantityordered") * col("Price"))
total_revenue = df2.select(sum("Revenue")).collect()[0][0]
print("Total revenue is: ", total_revenue)

Total revenue is:  8290886.790000005


In [0]:
df2.select('Quantityordered','Price','Revenue').show()

+---------------+-----+------------------+
|Quantityordered|Price|           Revenue|
+---------------+-----+------------------+
|             30| 95.7|            2871.0|
|             34|81.35|2765.8999999999996|
|             41|94.74|3884.3399999999997|
|             45|83.26|3746.7000000000003|
|             49|100.0|            4900.0|
|             36|96.66|3479.7599999999998|
|             29|86.13|           2497.77|
|             48|100.0|            4800.0|
|             22|98.57|           2168.54|
|             41|100.0|            4100.0|
|             37|100.0|            3700.0|
|             23|100.0|            2300.0|
|             28|100.0|            2800.0|
|             34|100.0|            3400.0|
|             45|92.83|           4177.35|
|             36|100.0|            3600.0|
|             23|100.0|            2300.0|
|             41|100.0|            4100.0|
|             46|94.74|           4358.04|
|             42|100.0|            4200.0|
+----------

In [0]:
# calculating the average revenue per transaction.
avg_revenue = df2.groupBy("Tr_ID").sum("Revenue").agg(avg("sum(Revenue)")).collect()[0][0]
print("Average revenue per transaction is: ", avg_revenue)

Average revenue per transaction is:  2072721.6974999988


In [0]:
df2.orderBy('ORDERNUMBER').show()

+-----------+---------------+-----+---------------+----------------+-------+-----+--------+----+----------------+----+-----------+--------------------+-----------------+--------------------+------------+---------+-----+----------+-------+---------+---------------+----------------+--------+------------------+
|ORDERNUMBER|QUANTITYORDERED|PRICE|ORDERLINENUMBER|       ORDERDATE| STATUS|TR_ID|MONTH_ID|YEAR|     PRODUCTLINE|MSRP|PRODUCTCODE|        CUSTOMERNAME|            PHONE|        ADDRESSLINE1|ADDRESSLINE2|     CITY|STATE|POSTALCODE|COUNTRY|TERRITORY|CONTACTLASTNAME|CONTACTFIRSTNAME|DEALSIZE|           Revenue|
+-----------+---------------+-----+---------------+----------------+-------+-----+--------+----+----------------+----+-----------+--------------------+-----------------+--------------------+------------+---------+-----+----------+-------+---------+---------------+----------------+--------+------------------+
|      10100|             30|100.0|              3|01-06-2003 00:00|Sh

In [0]:
# identify the top-selling products
top_products = df.groupBy("Productcode").sum("QUANTITYORDERED").sort(desc("sum(QUANTITYORDERED)")).limit(10).collect()
print("Top selling products: ")
for product in top_products:
    print(product.Productcode)

Top selling products: 
S18_3232
S24_3856
S18_4600
S700_4002
S12_4473
S24_3949
S18_1097
S50_1341
S18_2432
S18_1342
